# EEEL 4220 HW6
## Fall 2022

In this homework you will execute and modify a given unit commitment code to analyze system costs. Please **separate** your answers and your code, only write down necessary equations, analysis, and results in the answer to the question. Attach your code to the end of the notebook in the appendix.

A utility has three generators:


| Generator | MC [\\$/MWh] | NLC [\\$/h]   | SC [\\$]  |  GenMin [MW]  |  GenMax [MW] | Initial Status |
| :- | -: | -: | -: | -: | -: | -:|
| A | 20      |    500    |   5000    | 200 | 500  | On |
| B | 50      |    300    |   2000    | 50 | 200  | Off |
| C | 30      |    200    |   3000    | 100 | 300  | Off |

Consider the following load curve

|Hour | Demand [MW] |
| :- | -: | 
|1|  450 |
|2 | 525 |
|3 | 765 |
|4 | 525 |

And this problem can be formulated as a mixed-integer linear programming as
$$
\begin{align}
    \min_{g,u,v} \sum_{t=1}^4H_t\sum_{i=1}^3 \Big(\mathrm{MC_i}g_{i,t} + \mathrm{NLC}_iu_{i,t}\Big) + \sum_{t=1}^4\sum_{i=1}^3 \mathrm{SUC}_i v_{i,t}
\end{align}
$$
subjects to
$$
\begin{gather}
    \sum_{i=1}^3 g_{i,t} = \mathrm{Demand}_t\\
    \mathrm{GenMin}_i u_{i,t} \leq g_{i,t} \leq \mathrm{GenMax}_i u_{i,t} \\
    v_{i,t} \geq u_{i,t} - u_{i,t-1} \\
    v_{i,t}\in\{0,1\},\quad u_{i,t}\in\{0,1\}\quad (\text{$u_{i,t}$ and $v_{i,t}$ are binaries})
\end{gather}
$$
where $g_{i,t}$ is the dispatch results for generator $i$ during period $t$, $u_{i,t}$ is the commitment status, $v_{i,t}$ is the start-up status, $H_t$ is the duration of each period.

Please use and modify the attached sample unit commitment code and finish the following exercises. Note that operating cost includes fuel cost, start-up cost, and no-load cost. In each question, mention how you modified the sample code, list and discuss results.

### Problem 1 Run unit commitments (15)

Solve the unit commitment problem using the attached code. What is the total operating cost? (10 pts)

<mark> The total operating cost is $55550.0, based on the calculation from the code. 
    

In [ ]:
import cvxpy as cp
import numpy as np
#import gurobipy 


# Print the installed solver to see if GUROBI is installed correctly
print("Current installed solvers: ", cp.installed_solvers())

# Define parameters
N = 3; # number of generators
H = 4; # number of hours

# unit parameters are 1 by 3 vectors for the three generators
MC      = np.array([20, 50, 30])   # marginal generator cost
NLC     = np.array([500,300,200])  # no load cost
SUC     = np.array([5000, 2000, 3000])  # start up cost
GenMin  = np.array([200,50,100])   # minium generation
GenMax  = np.array([500,200,300])  # maximum generation
u0      = np.array([1,0,0])        # initial commitment status


LOAD = np.array([450,525,765,525]);

# the first index is generator, the second is period
g = cp.Variable((N,H), nonneg = True)   # generator dispatch
u = cp.Variable((N,H), boolean = True)   # commitment status
v = cp.Variable((N,H), boolean = True)   # start-up status

obj = cp.Minimize( sum(MC @ g + NLC @ u)  + sum(SUC @ v))

# Initialize an empty constraint set
con_set_1 = []  

# power balance constraint, supply equals demand
con_set_1.append( LOAD == sum(g) ) # demand balance constraint

# use a for loop to define the unit constriant over each time period
for t in range(H):  # go through each period
    for i in range(N):  # go through each  generator
        con_set_1.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
        con_set_1.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
        
        if t==0:
            # for the first period, check with initial commitment status
            con_set_1.append( v[i][0] >= u[i][0] - u0[i]) 
        else:
            # for other periods, check difference between two commitment status
            con_set_1.append( v[i][t] >= u[i][t] - u[i][t-1])
        
# Solve the problem
prob1 = cp.Problem(obj, con_set_1)
#prob1.solve(solver = 'GUROBI')
prob1.solve();

import pandas
# define row names and headers
row_Names = ['Unit1', 'Unit2', 'Unit3']
col_Names = ['Hour 1', 'Hour 2', 'Hour 3', 'Hour 4']

# print out results
print("\n Total operating cost: %.1f" % obj.value)

print("\n Generator dispatch results: ")
GT = pandas.DataFrame(g.value,  row_Names, col_Names)
print(GT.round(1))

print("\n Generator commitment results: ")
UT = pandas.DataFrame(u.value,  row_Names, col_Names)
print(UT.round(1))

print("\n Generator start-up results: ")
VT = pandas.DataFrame(v.value,  row_Names, col_Names)
print(VT.round(1))

Current installed solvers:  ['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

 Total operating cost: 55550.0

 Generator dispatch results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1   450.0   425.0   500.0   425.0
Unit2    -0.0    -0.0    -0.0    -0.0
Unit3     0.0   100.0   265.0   100.0

 Generator commitment results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     1.0     1.0     1.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     1.0     1.0     1.0

 Generator start-up results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     0.0     0.0     0.0     0.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     1.0     0.0     0.0


### Problem 2 Get marginal prices (15)


You may have noticed that the MILP does not provide you value for dual variables, as the problem is not convex and Gurobi deemed dual variables are not legit in this problem. To get the marginal prices, please **duplicate** the code (do not modify it), replace `u` and `v` with `u.value` and `v.value` everwhere in the code. This converts the problem into a LP by fixing all binary decisions based on the MILP result.

Run the LP version, your code should produce exactly the same result to MILP, but you should get the dual variables now to the demand balance constraint, which is the marginal prices.

List the marginal price results for each hour. 

In [ ]:
#QUESTION 2 CODE 

obj2 = cp.Minimize( sum(MC @ g + NLC @ u.value)  + sum(SUC @ v.value))

# Initialize an empty constraint set
con_set_2 = []  

# power balance constraint, supply equals demand
con_set_2.append( LOAD == sum(g) ) # demand balance constraint

# use a for loop to define the unit constriant over each time period
for t in range(H):  # go through each period
    for i in range(N):  # go through each  generator
        con_set_2.append(g[i][t] <= GenMax[i] * u.value[i][t])  # maximum generation limits
        con_set_2.append(g[i][t] >= GenMin[i] * u.value[i][t])  # minimum generation limits
        
        
# Solve the problem
prob2 = cp.Problem(obj2, con_set_2)
prob2.solve(solver = 'GUROBI')
prob2.solve();

print("\n Marginal Price: ")
MP = pandas.DataFrame(-con_set_2[0].dual_value, col_Names)
print(MP.round(1))


 Marginal Price: 
           0
Hour 1  20.0
Hour 2  20.0
Hour 3  30.0
Hour 4  20.0


### Problem 3 Revenue adaquecy (10)

Assume the utility charges consumer using the marginal generation cost, how much revenue will the utility company collect? Was the utility able to profit? Why? (10 pts)

Revenue = cost x quantity <br>

For each hour choose marginal cost: <br>
20 x 450 + 525 x 20 + 765 x 30 + 525 x 20 = 59,250<br>
Profit = revenue - cost <br>
59,250 - 55,550 = 3,700<br>

<mark> The company is not profitable, with $-2,600 in losses. <br>

In [ ]:
cost = 55550.0
revenue = 20 * 450 + 525 * 20 + 765 * 30 + 525 * 20
profit = revenue - cost
print("The company's profit is negative", profit)

The company's profit is negative -2600.0


### Problem 4 Ramp rates (15)

Now assume each unit has a 100~MW/h ramp rate. What is the total operating cost now? What is different now from the case without ramp limits? (10 pts)

<mark> From the updated code, we now have a higher operating cost of $59150.0.



In [ ]:
obj4 = cp.Minimize( sum(MC @ g + NLC @ u)  + sum(SUC @ v))

# Initialize an empty constraint set
con_set_4 = []  

# power balance constraint, supply equals demand
con_set_4.append( LOAD == sum(g) ) # demand balance constraint

# use a for loop to define the unit constriant over each time period
for t in range(H):  # go through each period
    for i in range(N):  # go through each  generator
        con_set_4.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
        con_set_4.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
        
        if t==0:
            # for the first period, check with initial commitment status
            con_set_4.append( v[i][0] >= u[i][0] - u0[i]) 
        else:
            # for other periods, check difference between two commitment status
            con_set_4.append( v[i][t] >= u[i][t] - u[i][t-1])
            con_set_4.append( g[i][t] - g[i][t-1] <= 100) #ramp-up
            con_set_4.append( g[i][t-1] - g[i][t] >= -100) #ramp-down

# Solve the problem
prob4 = cp.Problem(obj4, con_set_4)
prob4.solve(solver = 'GUROBI')
prob4.solve();

import pandas
# define row names and headers
row_Names = ['Unit1', 'Unit2', 'Unit3']
col_Names = ['Hour 1', 'Hour 2', 'Hour 3', 'Hour 4']

# print out results
print("\n Total operating cost: %.1f" % obj4.value)

print("\n Generator dispatch results: ")
GT = pandas.DataFrame(g.value,  row_Names, col_Names)
print(GT.round(1))

print("\n Generator commitment results: ")
UT = pandas.DataFrame(u.value,  row_Names, col_Names)
print(UT.round(1))

print("\n Generator start-up results: ")
VT = pandas.DataFrame(v.value,  row_Names, col_Names)
print(VT.round(1))


 Total operating cost: 59150.0

 Generator dispatch results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1   450.0   425.0   500.0   425.0
Unit2     0.0     0.0    65.0     0.0
Unit3     0.0   100.0   200.0   100.0

 Generator commitment results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     1.0     1.0     1.0
Unit2     0.0     0.0     1.0     0.0
Unit3     0.0     1.0     1.0     1.0

 Generator start-up results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     0.0     0.0     0.0     0.0
Unit2     0.0     0.0     1.0     0.0
Unit3     0.0     1.0     0.0     0.0


### Probelm 5 Reserve (15)

Remove the ramp rates, now assume the system operator also requires a 10% reserve over the demand. For example, in the first hour, the demand is 450 then the reserve requirement will be 45. What is the total generator cost of providing reserve in this system?

<mark> Adding 10% to every hour demand, the new total generating cost is: 64,145.0 and we see that we have <br> generator 3 also running during hour 1 , 2 and 3. Also, in order to have the reserve we're going to have to run<br> additionally generator 2 during the third hour. <br>
Generator dispatch results: <br> 
       Hour 1  Hour 2  Hour 3  Hour 4<br> 
Unit1   495.0   477.5   500.0   477.5<br> 
Unit2     0.0     0.0    50.0     0.0<br> 
Unit3     0.0   100.0   291.5   100.0<br> 
The new cost is now 52,550 without startup costs and 57,550 with the startup cost (not inlcuding gen 1) <br>  <mark>

In [ ]:
obj5 = cp.Minimize( sum(MC @ g + NLC @ u)  + sum(SUC @ v))

# Initialize an empty constraint set
con_set_5 = []  
u2 = np.array([1,0,1]) 
u3 = np.array([1,1,1])

# power balance constraint, supply equals demand
con_set_5.append( LOAD == sum(g) ) # demand balance constraint


# use a for loop to define the unit constriant over each time period
for t in range(H):  # go through each period
    for i in range(N):  # go through each  generator
        con_set_5.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
        con_set_5.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
        
        if t==0 :
            con_set_5.append( v[i][0] >= u0[i])
        if (t==1) or (t == 3):
            con_set_5.append( v[i][0] >= u2[i])
        if t==2: 
            con_set_5.append( v[i][0] >= u3[i])

# Solve the problem
prob5 = cp.Problem(obj5, con_set_5)
prob5.solve(solver = 'GUROBI')
prob5.solve();

import pandas
# define row names and headers
row_Names = ['Unit1', 'Unit2', 'Unit3']
col_Names = ['Hour 1', 'Hour 2', 'Hour 3', 'Hour 4']

# print out results
print("\n Total operating cost: %.1f" % obj5.value)

print("\n Generator dispatch results: ")
GT = pandas.DataFrame(g.value,  row_Names, col_Names)
print(GT.round(1))

print("\n Generator commitment results: ")
UT = pandas.DataFrame(u.value,  row_Names, col_Names)
print(UT.round(1))

print("\n Generator start-up results: ")
VT = pandas.DataFrame(v.value,  row_Names, col_Names)
print(VT.round(1))


 Total operating cost: 62550.0

 Generator dispatch results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1   450.0   425.0   500.0   425.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0   100.0   265.0   100.0

 Generator commitment results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     1.0     1.0     1.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     1.0     1.0     1.0

 Generator start-up results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     0.0     0.0     0.0
Unit2     1.0     0.0     0.0     0.0
Unit3     1.0     0.0     0.0     0.0


### Probelm 6 Reserve payments (10)

Following previous problem, assume consumers still pay electricity at the marginal price, how much additional payment are consumers paying for reserve?

20 x (495-450) + (577.5-525) x 20 + (841.5-765) x 30 + (577.5-525) x 20 = 5,295 <br>
<mark> The consumers are paying total 5,295 for the reserve, if the electricity is being generated. However, <br>
for the service of having the reserve as option, the consumers are paying additional 57,550 - 55,550 = $2,000 <mark>



In [ ]:
add_pay = 20*(495-450) + (577.5-525)*20 + (841.5-765)*30 + (577.5-525)*20
print("The additional pay is", add_pay)

The additional pay is 5295.0


### Problem 7 Reduced demand (10)

Neglect reserve and ramp rates, perform the unit commitment again with 80% system demand over all hours (i.e., for the first period the demand changes to 360~MW), what is the total operating cost now? (10 pts)

<mark> The new calculated total operating cost is $42,560.0

In [ ]:
obj7 = cp.Minimize( sum(MC @ g + NLC @ u)  + sum(SUC @ v))

# Initialize an empty constraint set
con_set_7 = []  

# power balance constraint, supply equals demand
con_set_7.append( LOAD*0.8 == sum(g) ) # demand balance constraint

# use a for loop to define the unit constriant over each time period
for t in range(H):  # go through each period
    for i in range(N):  # go through each  generator
        con_set_7.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
        con_set_7.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
        
        if t==0:
            # for the first period, check with initial commitment status
            con_set_7.append( v[i][0] >= u[i][0] - u0[i]) 
        else:
            # for other periods, check difference between two commitment status
            con_set_7.append( v[i][t] >= u[i][t] - u[i][t-1])

# Solve the problem
prob7 = cp.Problem(obj7, con_set_7)
prob7.solve(solver = 'GUROBI')
prob7.solve();

import pandas
# define row names and headers
row_Names = ['Unit1', 'Unit2', 'Unit3']
col_Names = ['Hour 1', 'Hour 2', 'Hour 3', 'Hour 4']

# print out results
print("\n Total operating cost: %.1f" % obj7.value)

print("\n Generator dispatch results: ")
GT = pandas.DataFrame(g.value,  row_Names, col_Names)
print(GT.round(1))

print("\n Generator commitment results: ")
UT = pandas.DataFrame(u.value,  row_Names, col_Names)
print(UT.round(1))

print("\n Generator start-up results: ")
VT = pandas.DataFrame(v.value,  row_Names, col_Names)
print(VT.round(1))


 Total operating cost: 42560.0

 Generator dispatch results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1   360.0   420.0   500.0   420.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     0.0   112.0     0.0

 Generator commitment results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     1.0     1.0     1.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     0.0     1.0     0.0

 Generator start-up results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     0.0     0.0     0.0     0.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     0.0     1.0     0.0


### Probelm 8 Real-time operation (10)

Assume the utility company already scheduled the commitment status with 100\% load, but during the day system demand dropped to 80%. Hence the utility cannot change the on/off status of generators, but can adjust the generation of started units. What is the total operating cost? Is it different from the previous question? Why? (10 pts)
  
<mark>   
The new operating cost is: $47560.0 <br> 
Therefore, our new operating cost compared to question 7, has increased, because we have to keep our first and third generators running during hour 1, 2, and 3 even if they might not produce anything (that is based of the load distribution generation from question 1, and the load generation in question 7). <mark>

In [ ]:
obj8 = cp.Minimize( sum(MC @ g + NLC @ u)  + sum(SUC @ v))

u1 = np.array([1,0,1])

# Initialize an empty constraint set
con_set_8 = []  

# power balance constraint, supply equals demand
con_set_8.append( LOAD*0.8 == sum(g) ) # demand balance constraint

# use a for loop to define the unit constriant over each time period
for t in range(H):  # go through each period
    for i in range(N):  # go through each  generator
        con_set_8.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
        con_set_8.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
        
        if t==0 :
            con_set_8.append( v[i][0] >= u0[i])
        if (t==1) or (t == 2) or (t == 3):
            con_set_8.append( v[i][0] >= u1[i])
       
            
# Solve the problem
prob8 = cp.Problem(obj8, con_set_8)
prob8.solve(solver = 'GUROBI')
prob8.solve();

import pandas
# define row names and headers
row_Names = ['Unit1', 'Unit2', 'Unit3']
col_Names = ['Hour 1', 'Hour 2', 'Hour 3', 'Hour 4']

# print out results
print("\n Total operating cost: %.1f" % obj8.value)

print("\n Generator dispatch results: ")
GT = pandas.DataFrame(g.value,  row_Names, col_Names)
print(GT.round(1))

print("\n Generator commitment results: ")
UT = pandas.DataFrame(u.value,  row_Names, col_Names)
print(UT.round(1))

print("\n Generator start-up results: ")
VT = pandas.DataFrame(v.value,  row_Names, col_Names)
print(VT.round(1))


 Total operating cost: 47560.0

 Generator dispatch results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1   360.0   420.0   500.0   420.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     0.0   112.0     0.0

 Generator commitment results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     1.0     1.0     1.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     0.0     1.0     0.0

 Generator start-up results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     0.0     0.0     0.0
Unit2     0.0     0.0     0.0     0.0
Unit3     1.0     0.0     0.0     0.0


### Appendix: Unit Commitment Code

Please do not use this code as your answers to Problem 2. Use the code to generate results such as system prices and generation outputs, and use them to answer Problem 2. The code should be kept here as an appendix.

We need two packages. `cvxpy` for formulating the optimization problem. `numpy` for doing matrix calculation. We will also need Gurobi to solve MILP. 

#### Install Gurobi

* First, register a [Gurobi][1] account using your Columbia email. 
* Log-in into your Gurobi account, obtain an academic license under Academia. 
* Install the newest version of Gurobi optimizer.
* Start termianl or command line (don't use Conda environment for this, start the terminal directly from your main system), and activate your Gurobi installation per instructions on your license page. **NOTE:** Gurobi must be activated either via Columbia network on campus or through Columbia VPN if you are off-campus. 


[1]: https://www.gurobi.com/


**First, restart your Jupyter Notebook after setting up Gurobi.**

Execute the following command in terminal to install the ``gurobipy`` package that supports Gurobi in Python
>~~~python
>pip install gurobipy
>~~~

Then execute the following comment and you should see Gurobi included in the installed solvers.


In [ ]:
%pip install gurobipy 
import cvxpy as cp
import numpy as np
import gurobipy 

# Print the installed solver to see if GUROBI is installed correctly
print("Current installed solvers: ", cp.installed_solvers())

Note: you may need to restart the kernel to use updated packages.
Current installed solvers:  ['ECOS', 'ECOS_BB', 'GUROBI', 'OSQP', 'SCIPY', 'SCS']


#### Solve the unit commitment

The first step is to define parameters in our optimizaiton problem. Here we use matrix forms and define all parameters as row vectors.

In [ ]:
# Define parameters
N = 3; # number of generators
H = 4; # number of hours

# unit parameters are 1 by 3 vectors for the three generators
MC      = np.array([20, 50, 30])   # marginal generator cost
NLC     = np.array([500,300,200])  # no load cost
SUC     = np.array([5000, 2000, 3000])  # start up cost
GenMin  = np.array([200,50,100])   # minium generation
GenMax  = np.array([500,200,300])  # maximum generation
u0      = np.array([1,0,0])        # initial commitment status


LOAD = np.array([450,525,765,525]);

The next step is to define variables as 3 by 4 matrices - 3 generators across 4 time periods.

In [ ]:
# the first index is generator, the second is period
g = cp.Variable((N,H), nonneg = True)   # generator dispatch
u = cp.Variable((N,H), boolean = True)   # commitment status
v = cp.Variable((N,H), boolean = True)   # start-up status

Define the objective function using matrix multiplications `@`

In [ ]:
obj = cp.Minimize( sum(MC @ g + NLC @ u)  + sum(SUC @ v))

Define constraints

$$
\begin{gather}
    \sum_{i=1}^3 g_{i,t} = \mathrm{Demand}_t\\
    \mathrm{GenMin}_i u_{i,t} \leq g_{i,t} \leq \mathrm{GenMax}_i u_{i,t} \\
    v_{i,t} \geq u_{i,t} - u_{i,t-1} 
\end{gather}
$$

In [ ]:
# Initialize an empty constraint set
con_set_1 = []  

# power balance constraint, supply equals demand
con_set_1.append( LOAD == sum(g) ) # demand balance constraint

# use a for loop to define the unit constriant over each time period
for t in range(H):  # go through each period
    for i in range(N):  # go through each  generator
        con_set_1.append(g[i][t] <= GenMax[i] * u[i][t])  # maximum generation limits
        con_set_1.append(g[i][t] >= GenMin[i] * u[i][t])  # minimum generation limits
        
        if t==0:
            # for the first period, check with initial commitment status
            con_set_1.append( v[i][0] >= u[i][0] - u0[i]) 
        else:
            # for other periods, check difference between two commitment status
            con_set_1.append( v[i][t] >= u[i][t] - u[i][t-1])

            

Solve the optimization problem with the objective function `obj` and constraints `constraints`

In [ ]:
# Solve the problem
prob1 = cp.Problem(obj, con_set_1)
prob1.solve(solver = 'GUROBI')
prob1.solve();

Print out the results. To print out some nice tables, use `pandas` package. You need to install it from terminal using
`pip install pandas`

In [ ]:
import pandas

# define row names and headers
row_Names = ['Unit1', 'Unit2', 'Unit3']
col_Names = ['Hour 1', 'Hour 2', 'Hour 3', 'Hour 4']

In [ ]:
# print out results
print("\n Total operating cost: %.1f" % obj.value)

print("\n Generator dispatch results: ")
GT = pandas.DataFrame(g.value,  row_Names, col_Names)
print(GT.round(1))

print("\n Generator commitment results: ")
UT = pandas.DataFrame(u.value,  row_Names, col_Names)
print(UT.round(1))

print("\n Generator start-up results: ")
VT = pandas.DataFrame(v.value,  row_Names, col_Names)
print(VT.round(1))


 Total operating cost: 55550.0

 Generator dispatch results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1   450.0   425.0   500.0   425.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0   100.0   265.0   100.0

 Generator commitment results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     1.0     1.0     1.0     1.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     1.0     1.0     1.0

 Generator start-up results: 
       Hour 1  Hour 2  Hour 3  Hour 4
Unit1     0.0     0.0     0.0     0.0
Unit2     0.0     0.0     0.0     0.0
Unit3     0.0     1.0     0.0     0.0
